In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
File: s2_test.py
Version: v1.0
Date: 2023-02-09
Authors: Chen G.
Description: This script creates downloading and processing Sentinel-2 images based on Google Earth Engine.
License: This code is distributed under the MIT License.

    Parameter:
        START_DATE: The earliest date to include images for (inclusive).
        END_DATE: The latest date to include images for (exclusive).
        BANDS: The Sentinel-2 image bands to select for processing.
        ROI: The boundry to select for processing.
        MAX_CLOUD_PROBABILITY: (Optional) cloud cover percentage to apply s2 image collection filter.
        CAL_NDVI: (Optional) calculate the Normalized Difference Vegetation Index (NDVI) from multiband s2 images.
        CAL_NDMI: (Optional) calculate the Normalized Difference Moisture Index (NDMI) from multiband s2 images.
        CLIP_TO_ROI: (Optional) clip the processed image to the region of interest.
        SAVE_ASSETS : (Optional) exports the processed collection to an asset.
        ASSET_ID : (Optional) the user id path to save the assets
        SAVE_LOCAL : (Optional) download the processed images to local.
        VISUALIZATION : (Optional) convert raw image to RGB image and download the processed images to local.
        LOCAL_DIR : (Optional) where to save downloaded images.
        
    Returns:
        An ee.ImageCollection with an analysis ready Sentinel 2 imagery with the cloud masked images and vegetation index band.


    """

import base_process as bp


# Query Dates
dates_list_2019 = ['2018-12-29', '2018-12-31', '2019-01-23', '2019-03-11', '2019-03-14', '2019-03-31', '2019-04-13', '2019-04-18', 
                   '2019-05-03', '2019-05-10', '2019-05-23', '2019-06-02', '2019-06-04', '2019-06-07', '2019-06-09', '2019-06-12', 
                   '2019-06-14', '2019-06-22', '2019-06-24', '2019-06-27', '2019-07-02', '2019-07-07', '2019-07-14', '2019-07-27', 
                   '2019-07-29', '2019-08-01', '2019-08-06', '2019-08-16', '2019-08-18', '2019-08-23', '2019-08-28', '2019-08-31', 
                   '2019-09-05', '2019-09-07', '2019-09-20', '2019-09-22', '2019-09-25', '2019-09-27', '2019-09-30', '2019-10-02', 
                   '2019-10-15', '2019-10-17', '2019-10-20', '2019-10-30', '2019-11-06', '2019-11-11', '2019-11-14', '2019-11-16', 
                   '2019-11-19', '2019-12-04', '2019-12-09', '2019-12-29', '2019-12-31']

dates_list_2020 = ['2020-01-28', '2020-02-09', '2020-02-12', '2020-02-17', '2020-02-22', '2020-03-15', '2020-03-18', '2020-03-20', 
                   '2020-03-23', '2020-03-28', '2020-04-07', '2020-04-12', '2020-04-22', '2020-04-27', '2020-04-29', '2020-05-02', 
                   '2020-05-04', '2020-05-12', '2020-05-17', '2020-05-19', '2020-05-22', '2020-05-27', '2020-06-01', '2020-06-06',
                   '2020-06-08', '2020-07-06', '2020-07-08', '2020-08-10', '2020-08-12', '2020-08-15', '2020-08-17', '2020-08-25', 
                   '2020-08-27', '2020-08-30', '2020-09-01', '2020-09-04', '2020-09-06', '2020-09-11', '2020-09-19', '2020-09-24', 
                   '2020-09-26', '2020-10-21', '2020-10-24', '2020-10-29', '2020-11-08', '2020-11-10', '2020-11-13', '2020-11-20', 
                   '2020-12-05', '2020-12-15', '2020-12-30']

dates_list_2021 = ['2021-01-02', '2021-01-07', '2021-01-09', '2021-01-12', '2021-01-14', '2021-01-17', '2021-01-19', '2021-02-11', 
                   '2021-02-16', '2021-02-21', '2021-03-25', '2021-03-28', '2021-04-04', '2021-04-17', '2021-04-19', '2021-04-29', 
                   '2021-05-02', '2021-05-04', '2021-05-09', '2021-05-22', '2021-05-24', '2021-05-29', '2021-06-06', '2021-06-08', 
                   '2021-06-11', '2021-06-23', '2021-07-01', '2021-07-08', '2021-07-13', '2021-07-31', '2021-08-17', '2021-08-25', 
                   '2021-09-11', '2021-09-14', '2021-09-16', '2021-09-21', '2021-09-29', '2021-10-01', '2021-10-04', '2021-10-24', 
                   '2021-11-05', '2021-11-08', '2021-11-10', '2021-11-13', '2021-11-15', '2021-11-18', '2021-11-23', '2021-11-25', 
                   '2021-11-30', '2021-12-03', '2021-12-05', '2021-12-18', '2021-12-28', '2021-12-30']

dates_list_2022 = ['2022-02-01', '2022-02-03', '2022-02-21', '2022-02-26', '2022-03-03', '2022-03-05', '2022-03-08', '2022-03-18',  
                   '2022-03-23', '2022-03-28', '2022-04-02', '2022-04-07', '2022-04-09', '2022-04-17', '2022-04-22', '2022-04-27', 
                   '2022-05-02', '2022-05-04', '2022-05-17', '2022-05-24', '2022-05-29', '2022-06-01', '2022-06-06', '2022-06-11', 
                   '2022-06-13', '2022-06-16', '2022-06-18', '2022-06-28', '2022-07-01', '2022-07-08', '2022-07-13', '2022-07-18', 
                   '2022-07-21', '2022-08-05', '2022-08-07', '2022-08-12', '2022-08-15', '2022-08-20', '2022-09-01', '2022-09-04', 
                   '2022-09-06', '2022-09-11', '2022-09-19', '2022-09-26', '2022-09-29', '2022-10-09', '2022-10-11', '2022-10-14', 
                   '2022-10-16', '2022-10-19', '2022-10-21', '2022-10-24', '2022-10-29', '2022-10-31', '2022-11-05', '2022-11-10', 
                   '2022-11-15', '2022-11-20', '2022-11-25', '2022-12-03', '2022-12-05', '2022-12-18', '2022-12-23', '2022-12-25']

dates_list_2023 = ['2023-01-07', '2023-01-24', '2023-01-27', '2023-01-29', '2023-02-03', '2023-02-21', '2023-02-26', '2023-03-03',  
                   '2023-03-05', '2023-03-08', '2023-03-10', '2023-03-13', '2023-03-28', '2023-03-30', '2023-04-02', '2023-04-17', 
                   '2023-04-19']


for d in dates_list_2019:
    # processed s2 collection
    bp.dancheng_preprocess(d, "EPSG:4326", "G:/geoserver_data/ndvi_wgs84", "G:/geoserver_data/ndvi_gcj02")


dancheng_DT_ZS_20181229.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

dancheng_DT_ZS_20181231.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

dancheng_DT_ZS_20190123.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

dancheng_DT_ZS_20190311.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

dancheng_DT_ZS_20190314.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

...

dancheng_DT_ZS_20191119.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

dancheng_DT_ZS_20191204.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

dancheng_DT_ZS_20191209.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

dancheng_DT_ZS_20191229.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)

dancheng_DT_ZS_20191231.tif: |          | 0.00/890k (raw) [  0.0%] in 00:00 (eta:     ?)